# Signal vs. Noise – Simulation Demo

In this notebook, we simulate a signal+noise dataset and demonstrate how noise affects our ability to detect a true effect using a General Linear Model (GLM). We then apply a simple denoising method and re-fit the GLM.

## 1. Simulate signal + noise

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

np.random.seed(42)

# Parameters
n = 200
X = np.zeros(n)
X[100:] = 1  # Step function: signal starts halfway through
beta = 1.5   # true effect size

# Generate signal
signal = beta * X

# Add moderate noise
noise = np.random.normal(0, 1, n)
Y = signal + noise

# Visualize
plt.plot(Y, label='Observed')
plt.plot(signal, label='True Signal', linestyle='--')
plt.xlabel('Time'); plt.ylabel('Response')
plt.legend(); plt.title('Signal + Moderate Noise')
plt.show()


## 2. Fit GLM to detect signal

In [ ]:

X_design = sm.add_constant(X)
model = sm.OLS(Y, X_design).fit()
print(model.summary())


## 3. Add stronger noise – effect becomes hard to detect

In [ ]:

# Add stronger noise
noise_heavy = np.random.normal(0, 4, n)
Y_noisy = signal + noise_heavy

plt.plot(Y_noisy, label='Observed (noisy)')
plt.plot(signal, label='True Signal', linestyle='--')
plt.xlabel('Time'); plt.ylabel('Response')
plt.legend(); plt.title('Signal + High Noise')
plt.show()

# Re-fit GLM
model_noisy = sm.OLS(Y_noisy, X_design).fit()
print(model_noisy.summary())


## 4. Denoising: Simple Moving Average

In [ ]:

# Moving average smoothing (window=5)
Y_denoised = pd.Series(Y_noisy).rolling(5, center=True).mean().fillna(method='bfill').fillna(method='ffill')

plt.plot(Y_denoised, label='Denoised')
plt.plot(signal, label='True Signal', linestyle='--')
plt.xlabel('Time'); plt.ylabel('Response')
plt.legend(); plt.title('After Simple Denoising')
plt.show()

# Re-fit GLM
model_denoised = sm.OLS(Y_denoised, X_design).fit()
print(model_denoised.summary())
